In [2]:
import numpy as np
import pandas as pd
import re
import numpy

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
pd_atp_matches_2016 = pd.read_csv('tennis_atp/atp_matches_2016.csv')
my_data_stats = pd.read_csv('data/match_stats_2016.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total',
           'ms_winner_total_points_won','ms_winner_total_points_total',
            'ms_winner_second_serve_points_total','ms_loser_second_serve_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
            'w_2ndTot','l_2ndTot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name','score',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon',
        'w_bpSaved','w_bpFaced',
        'l_bpSaved','l_bpFaced']
pd_atp_matches_2016=pd_atp_matches_2016.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches_2016=pd.merge(pd_atp_matches_2016,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches_2016=pd_atp_matches_2016.drop_duplicates(keep='first').reset_index(drop=True)


#calculate number of games
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

pd_atp_matches_2016=pd_atp_matches_2016.assign(TotGames=0)

#score to number of games
for i in range(0,len(pd_atp_matches_2016.index)):
    str1=pd_atp_matches_2016.loc[i, 'score']
    
    start=findOccurrences(str1, '(')
    end=findOccurrences(str1, ')')

    for k in range(0,len(start)):
        for l in range(0,end[k]-start[k]+1):
            str1=change_char(str1,start[k]+l,' ')

    number_of_games=0
    for j in range (0,len(list(filter(str.isdigit, str1)))):
        number_of_games=number_of_games+int(list(filter(str.isdigit, str1))[j])
    pd_atp_matches_2016.loc[i, 'TotGames'] = number_of_games

    
def SetNan(pandas,column):
    list = pandas.index[(pandas[column].isin([0])) == True].tolist()
    for i in range(0,len(list)):
        pandas.loc[list[i],column]=np.nan
        
SetNan(pd_atp_matches_2016,'w_svpt')
SetNan(pd_atp_matches_2016,'l_svpt')
SetNan(pd_atp_matches_2016,'w_1stIn')
SetNan(pd_atp_matches_2016,'l_1stIn')
SetNan(pd_atp_matches_2016,'w_2ndTot')
SetNan(pd_atp_matches_2016,'l_2ndTot')

In [4]:
#this function returns WRP of a player against a certain adversary

def getMatchesWonPer(myopponent,adversary):
    exp1=pd_atp_matches_2016['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2016['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2016['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2016['winner_id'].isin([adversary])

    matches_won=len(pd_atp_matches_2016.loc[(exp1&exp2)].index)
    matches_lost=len(pd_atp_matches_2016.loc[(exp3&exp4)].index)
    matchesWonPer = matches_won/(matches_won+matches_lost)
    
    return matchesWonPer

pd_atp_matches_2016=pd_atp_matches_2016.assign(TMW=0)
for i in range(0,len(pd_atp_matches_2016.index)):
    TMW1=getMatchesWonPer(pd_atp_matches_2016.loc[i, 'winner_id'],pd_atp_matches_2016.loc[i, 'loser_id'])
    pd_atp_matches_2016.loc[i, 'TMW'] = TMW1 - (1-TMW1)
    


In [6]:
display(pd_atp_matches_2016.loc[467])

tourney_id                2016-0495
tourney_name                  Dubai
round                            QF
winner_id                    106401
winner_name            Nick Kyrgios
loser_id                     104607
loser_name            Tomas Berdych
score                       6-4 6-4
winner_rank                      33
winner_rank_points             1240
loser_rank                        7
loser_rank_points              3990
w_ace                            10
l_ace                             4
w_df                              4
l_df                              4
w_svpt                           63
w_1stWon                         30
w_1stIn                          39
l_svpt                           67
l_1stWon                         29
l_1stIn                          35
w_2ndWon                         15
l_2ndWon                         13
w_bpSaved                         4
w_bpFaced                         4
l_bpSaved                         5
l_bpFaced                   

In [8]:
results2016 = pd_atp_matches_2016.filter(['winner_name','loser_name'], axis=1)

#Ranking Won
results2016['RANK'] = pd_atp_matches_2016['winner_rank'] - pd_atp_matches_2016['loser_rank']
results2016['POINTS'] = pd_atp_matches_2016['winner_rank_points'] - pd_atp_matches_2016['loser_rank_points']

#First Serve In
FS1 = pd_atp_matches_2016['w_1stIn'].div(pd_atp_matches_2016['w_svpt']) 
FS2 =  pd_atp_matches_2016['l_1stIn'].div(pd_atp_matches_2016['l_svpt'])
results2016['FS'] = FS1 - FS2

#Winning in First Serve
W1SP1 = pd_atp_matches_2016['w_1stWon'].div(pd_atp_matches_2016['w_1stIn'])
W1SP2 =  pd_atp_matches_2016['l_1stWon'].div(pd_atp_matches_2016['l_1stIn'])
results2016['W1SP'] = W1SP1 - W1SP2

#Winning in Second Serve
W2SP1 = pd_atp_matches_2016['w_2ndWon'].div(pd_atp_matches_2016['w_2ndTot'])
W2SP2 =  pd_atp_matches_2016['l_2ndWon'].div(pd_atp_matches_2016['l_2ndTot'])
results2016['W2SP'] = W2SP1 - W2SP2

#Winning on Serve Cumulative
WSP1 = W1SP1 * FS1 + W2SP1 * (1-FS1)
WSP2 = W1SP2 * FS2 + W2SP2 * (1-FS2)
results2016['WSP'] = WSP1 - WSP2

#Winning in Return
WRP1 = pd_atp_matches_2016['w_ret_w'].div(pd_atp_matches_2016['w_ret_tot'])
WRP2 =  pd_atp_matches_2016['l_ret_w'].div(pd_atp_matches_2016['l_ret_tot'])
results2016['WRP'] = WRP1 - WRP2

#Total Points Won
TPW1 = pd_atp_matches_2016['w_pts_w']/pd_atp_matches_2016['total_points']
TPW2 = (pd_atp_matches_2016['total_points']-pd_atp_matches_2016['w_pts_w'])/pd_atp_matches_2016['total_points']
results2016['TPW'] = TPW1 - (1-TPW1)

results2016['TMW'] = pd_atp_matches_2016['TMW']

results2016['ACES'] = (pd_atp_matches_2016['w_ace']-pd_atp_matches_2016['l_ace'])/pd_atp_matches_2016['TotGames']
results2016['DF'] = (pd_atp_matches_2016['w_df']-pd_atp_matches_2016['l_df'])/pd_atp_matches_2016['TotGames']

BP1 = pd_atp_matches_2016['w_bpSaved']/pd_atp_matches_2016['w_bpFaced']
BP2 = pd_atp_matches_2016['l_bpSaved']/pd_atp_matches_2016['l_bpFaced']
results2016['BP'] = BP1-BP2

COMPLETE1 = WSP1 * WRP1
COMPLETE2 = WSP2 * WRP2
results2016['COMPLETE'] = COMPLETE1-COMPLETE2

SERVEADV1 = WSP1 - WRP2
SERVEADV2 = WSP2 - WRP1
results2016['SERVEADV'] = SERVEADV1-SERVEADV2

#results.rename(columns{'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results2016=results2016.assign(winner=1)

results2016.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,winner_name,loser_name,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner
0,Milos Raonic,Roger Federer,11.0,-6095.0,0.00928962,0.0882353,0.019943,0.0606557,0.060656,0.057851,1.000000,-0.05,0.15,0.400000,0.0606557,0.121311,1
1,Roger Federer,Dominic Thiem,-17.0,6665.0,-0.012616,0.271207,0.170455,0.223377,0.223377,0.211538,-0.333333,0.235294,-0.235294,-0.333333,0.223377,0.446753,1
2,Milos Raonic,Bernard Tomic,-4.0,495.0,-0.022604,0.0570673,0.100000,0.068264,0.068264,0.055215,1.000000,0.115385,0,0.000000,0.068264,0.136528,1
3,Roger Federer,Grigor Dimitrov,-25.0,6905.0,0.0303522,0.123449,0.112903,0.126854,0.126854,0.080214,1.000000,-0.030303,-0.030303,-0.727273,0.126854,0.253707,1
4,Dominic Thiem,Marin Cilic,7.0,-805.0,0.12415,-0.043007,0.102699,0.0323129,0.032313,0.000000,1.000000,-0.16129,-0.0645161,-0.875000,0.0323129,0.0646259,1
5,Milos Raonic,Lucas Pouille,-64.0,1498.0,-0.0313488,0.163593,0.221739,0.174658,0.174658,0.120000,1.000000,0.3,0,-0.625000,0.174658,0.349315,1
9,Marin Cilic,Hyeon Chung,-38.0,1588.0,-0.0614828,0.120909,-0.032393,0.0427421,0.042742,0.038462,1.000000,0.52,0.04,0.000000,0.0427421,0.0854841,1
1000,Gilles Simon,Pablo Carreno Busta,-28.0,921.0,-0.0692686,-0.0724638,0.061086,-0.0235584,-0.023558,-0.006623,0.333333,-0.185185,0.037037,0.166667,-0.0235584,-0.0471167,1
1001,Andy Murray,Radek Stepanek,-146.0,7552.0,-0.060571,-0.000331016,0.074176,0.0123457,0.012346,0.039548,1.000000,0.241379,-0.103448,0.175000,0.0123457,0.0246914,1
1002,Borna Coric,Nicolas Almagro,-8.0,71.0,0.22398,0.130227,0.505051,0.319388,0.319388,0.280899,1.000000,0.0625,-0.0625,0.666667,0.319388,0.638776,1


In [10]:
#Now order according to ranking to see how good they are

features2016 = results2016.filter(['RANK','POINTS','FS','W1SP','W2SP','WSP','WRP','TPW','TMW','ACES','DF','BP','COMPLETE','SERVEADV','winner'], axis=1)

features2016=features2016.assign(player1=' ')
features2016=features2016.assign(player2=' ')

for i in range(0,len(features2016.index)):
    if(features2016.loc[i, 'RANK']<0):
        features2016.loc[i, 'player1']=results2016.loc[i,'winner_name']
        features2016.loc[i, 'player2']=results2016.loc[i,'loser_name']
    else:
        features2016.loc[i, 'player1']=results2016.loc[i,'loser_name']
        features2016.loc[i, 'player2']=results2016.loc[i,'winner_name']
        features2016.loc[i, 'RANK':'SERVEADV']=features2016.loc[i, 'RANK':'SERVEADV']*-1
        features2016.loc[i, 'winner']=0

features2016=features2016.fillna(0)        
features2016.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

features2016.to_csv('newfeatures2016.csv',header='false',index='false')
features2016=pd.read_csv('newfeatures2016.csv')
features2016.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,Unnamed: 0,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner,player1,player2
0,0,-11.0,6095.0,-0.009290,-0.088235,-0.019943,-0.060656,-0.060656,-0.057851,-1.000000,0.050000,-0.150000,-0.400000,-0.060656,-0.121311,0,Roger Federer,Milos Raonic
1,1,-17.0,6665.0,-0.012616,0.271207,0.170455,0.223377,0.223377,0.211538,-0.333333,0.235294,-0.235294,-0.333333,0.223377,0.446753,1,Roger Federer,Dominic Thiem
2,2,-4.0,495.0,-0.022604,0.057067,0.100000,0.068264,0.068264,0.055215,1.000000,0.115385,0.000000,0.000000,0.068264,0.136528,1,Milos Raonic,Bernard Tomic
3,3,-25.0,6905.0,0.030352,0.123449,0.112903,0.126854,0.126854,0.080214,1.000000,-0.030303,-0.030303,-0.727273,0.126854,0.253707,1,Roger Federer,Grigor Dimitrov
4,4,-7.0,805.0,-0.124150,0.043007,-0.102699,-0.032313,-0.032313,-0.000000,-1.000000,0.161290,0.064516,0.875000,-0.032313,-0.064626,0,Marin Cilic,Dominic Thiem
5,5,-64.0,1498.0,-0.031349,0.163593,0.221739,0.174658,0.174658,0.120000,1.000000,0.300000,0.000000,-0.625000,0.174658,0.349315,1,Milos Raonic,Lucas Pouille
9,9,-38.0,1588.0,-0.061483,0.120909,-0.032393,0.042742,0.042742,0.038462,1.000000,0.520000,0.040000,0.000000,0.042742,0.085484,1,Marin Cilic,Hyeon Chung
1000,1000,-28.0,921.0,-0.069269,-0.072464,0.061086,-0.023558,-0.023558,-0.006623,0.333333,-0.185185,0.037037,0.166667,-0.023558,-0.047117,1,Gilles Simon,Pablo Carreno Busta
1001,1001,-146.0,7552.0,-0.060571,-0.000331,0.074176,0.012346,0.012346,0.039548,1.000000,0.241379,-0.103448,0.175000,0.012346,0.024691,1,Andy Murray,Radek Stepanek
1002,1002,-8.0,71.0,0.223980,0.130227,0.505051,0.319388,0.319388,0.280899,1.000000,0.062500,-0.062500,0.666667,0.319388,0.638776,1,Borna Coric,Nicolas Almagro
